In [6]:
#collecting data
import numpy as np
import scipy.io
mat = scipy.io.loadmat('clipped.mat') #dataset_connected_NYCwhile tryin in terminal, start path with Documents

## Preprocessing data

In [7]:
friendship_new = mat['fren_new']
friendship_old = mat['fren_old']
selected_checkins = mat['smol_checkins']
selected_users_IDs = mat['smol_user_IDs']
#temp = mat['selected_venue_IDs']
#selected_venue_IDs = []
#for i in range(len(temp)):
#    temper = temp[i][0]
#    selected_venue_IDs.append(temper[0])
#print(len(selected_venue_IDs)) #sanity check
#print(len(temp)) #should be same as above
print(max(selected_users_IDs))
print(len(selected_users_IDs))
print(max(map(max, selected_checkins)))
print(len(selected_checkins))
print(max(map(max, friendship_new)))
print(len(friendship_new))
print(max(map(max, friendship_old)))
print(len(friendship_old))
#max value in friendships should be same or less than the length of user_IDs

[1057]
20
10223374
363
12
3
19
4


In [8]:
# 1. rebuild node index
offset1 = max(selected_checkins[:,0]);
dumy, dumy, n = np.unique(selected_checkins[:,1],return_index=True,return_inverse=True, axis=0)
selected_checkins[:,1] = n+offset1;
#print(selected_checkins[:,1])
#print(offset1)
#print(n)
offset2 = max(selected_checkins[:,1]);
dumy, dumy, n = np.unique(selected_checkins[:,2],return_index=True,return_inverse=True, axis=0)
selected_checkins[:,2] = n+offset2;
#print(selected_checkins[:,2])
#print(offset2)
#print(n)
offset3 = max(selected_checkins[:,2]);
dumy, dumy, n = np.unique(selected_checkins[:,3],return_index=True,return_inverse=True, axis=0)
selected_checkins[:,3] = n+offset3;
#print(selected_checkins[:,3])
#print(offset3)
#print(n)

num_node_total = max(map(max, selected_checkins)) #max of the entire matrix = 8117
#print(min(map(min, selected_checkins)))

In [9]:
# 2. prepare checkins per user (fast)
#user_chechkins is a cell in MATLAB; using Python List for the same
user_checkins = [[] for _ in range(selected_users_IDs.shape[0])]
user_checkins_counter=np.zeros((len(selected_users_IDs),1)).astype(np.int64)
ind = selected_checkins[:,0].argsort(axis=0)
temp_checkins = np.array([selected_checkins[i,:] for i in ind]) #will not exactly be similar to MATLAB op
u,m,n = np.unique(temp_checkins[:,0],return_index=True,return_inverse=True, axis=0)
m=m.reshape(-1,1)
u=np.array(u)
print(u.shape)
counters = np.vstack((m[1:],temp_checkins.shape[0]))-m #+1 is not there because indexing starts from 0
ini_val=0
for i in range(0,u.shape[0]):
    user_checkins[u[i]-1].append(temp_checkins[ini_val:ini_val+counters[i,0],:])
    user_checkins[u[i]-1]=np.array(user_checkins[u[i]-1])[0,:,:].T
    user_checkins[u[i]-1].astype(np.int64)
    ini_val=ini_val+counters[i,0]
    user_checkins_counter[u[i]-1]=counters[i].astype(np.int64)
print(user_checkins[-1].shape) #for debugging

print(np.count_nonzero(user_checkins_counter)) #for debugging
len(user_checkins_counter)


(20,)
(4, 48)
20


20

In [10]:
# 3. random walk
from scipy.sparse import csr_matrix
num_node = len(selected_users_IDs)
print(num_node)
network=csr_matrix((np.ones((len(friendship_old),)), (friendship_old[:,0]-1, friendship_old[:,1]-1)), shape=(num_node, num_node))
network=network+network.T
print(network)

node_list=[[] for _ in range(num_node)]
node_list_len = np.zeros((num_node,));
num_walk = 10
len_walk = 80
(indx,indy) = network.T.nonzero()
#for i in range(len(indx)):
#    indx[i]+=1
#    indy[i]+=1

temp, m, n = np.unique(indx,return_index=True,return_inverse=True, axis=0) #check m for debugging
print(temp)
print(m)
print(n)
m=np.array(m).reshape(-1,1)
node_list_len=np.vstack((m[1:],len(indx))) - m #+1 is not there because indexing starts from 0
ini_val=0
temp=np.array(temp)
node_list_len=node_list_len[:,0] #for indexing

for i in range(0,temp.shape[0]):
    node_list[temp[i]].append(indy[ini_val:ini_val+node_list_len[i]])
    ini_val=ini_val+node_list_len[i]
node_list #for debugging match with MATLAB
#node_list_len.shape = np.count_nonzero(node_list_len) has to be as written in MATLAB script


20
  (0, 1)	1.0
  (1, 0)	1.0
  (2, 2)	2.0
  (6, 11)	1.0
  (11, 6)	1.0
  (16, 18)	1.0
  (18, 16)	1.0
[ 0  1  2  6 11 16 18]
[0 1 2 3 4 5 6]
[0 1 2 3 4 5 6]


[[array([1], dtype=int32)],
 [array([0], dtype=int32)],
 [array([2], dtype=int32)],
 [],
 [],
 [],
 [array([11], dtype=int32)],
 [],
 [],
 [],
 [],
 [array([6], dtype=int32)],
 [],
 [],
 [],
 [],
 [array([18], dtype=int32)],
 [],
 [array([16], dtype=int32)],
 []]

In [11]:
# let's have a walk over social network (friendship)
import random
walks = np.zeros((num_walk*num_node,len_walk),dtype = np.int64);
for ww in range(num_walk):
    for ii in range(num_node):
        seq = np.zeros((len_walk,),dtype=int)
        seq[0] = ii
        current_e = ii
        for jj in range(len_walk-1):
            rand_ind = random.randint(0, node_list_len[seq[jj]]-1);
            tempvar = node_list[seq[jj]][0] 
            #print(node_list_len[seq[jj]])
            #print(rand_ind)
            #print(tempvar.shape)
            seq[jj+1] = tempvar[rand_ind];
        
        walks[ii+(ww-1)*num_node,:] = seq
        
#preprocessing for removing 0 quantities
for i in range(walks.shape[0]):
    for j in range(walks.shape[1]):
        walks[i,j] = walks[i,j] +1
#print(min(map(min, walks))) #should be 1 and 4024
print(walks)

IndexError: list index out of range

In [7]:
# 4. prepare negative sample table in advance (fast)
# social relationship
from collections import Counter
(dumy,r) = network.nonzero(); #MATLAB gives [y,x]; Python gives (x,y)
temptab = Counter(r) #len(temptab) = size(tab_degree,1) #Counter gives unique elements in r
tab2= [[] for _ in range(len(temptab))]

for i in range(len(temptab)):
    tab2[i].append(temptab[i])
tab2=np.array(tab2)[:,0] #when printing temptab, not in asc. order but here it becomes asc. wrt keys

tab1=np.unique(r)
tab3=[[] for _ in range(len(tab1))]
tot=sum(tab2)
for i in range(len(tab1)):
    tab3[i]=round((tab2[i]*100)/tot,4)
tab_degree=np.stack((tab1,tab2,tab3),axis=1) # equivalent of tab_degree = tabulate(r) in MATLAB

freq = np.array([round(np.power(i,0.75),4) for i in tab_degree[:,2]] )
den=float(sum(freq))
neg_sam_table_social = np.repeat((1+tab_degree[:,0]),np.around(1000000*freq/sum(freq)).astype(np.int64))
neg_sam_table_social=neg_sam_table_social.astype(np.int64) # unigram with 0.75 power
#neg_sam_table_social[740] check for debugging
del temptab,tab1,tab2,tab3,tab_degree,freq,den
max(neg_sam_table_social)

4024

In [8]:
#checkins: user, venue, time, semantic, with node type normalization for each node domain

neg_sam_table_mobility_norm = [[] for _ in range(4)]
for ii in range(len(neg_sam_table_mobility_norm)):
    tab1=np.array([elem for elem in range(max(temp_checkins[:,ii]))])#tab1 has all elements;even with frequency 0
    temptab = Counter(temp_checkins[:,ii]) #correct
    tab2= np.zeros((len(tab1),))
    for i in tab1:
        tab2[i]= temptab[i+1] #because in python, index starts from 0
    tab2 = np.array(tab2)
    tot=np.sum(tab2)
    tab3=np.zeros((len(tab1),))
    for i in range(len(tab1)):
        tab3[i]= np.round((tab2[i]*100)/tot,4) if (tot) else 0
    tab3=np.array(tab3)
    tab_degree=np.stack((tab1,tab2,tab3),axis=1)
    freq = np.array([np.round(np.power(i,0.75),4) for i in tab_degree[:,2]])
    den=float(sum(freq))
    ingoes = np.repeat((tab_degree[:,0]+1),np.around(100000*freq/sum(freq)).astype(np.int64))
    neg_sam_table_mobility_norm[ii].append(ingoes)

    del tab1,tab2,i,tab3,tab_degree,freq,ingoes,tot

neg_sam_table_mobility_norm = [np.array(x).astype(np.int64) for x in neg_sam_table_mobility_norm]

#neg_sam_table_mobility1 = neg_sam_table_mobility_norm[3][0]
#neg_sam_table_mobility1[5]
print(neg_sam_table_mobility_norm)

[array([[   1,    1,    1, ..., 4024, 4024, 4024]]), array([[4024, 4024, 4024, ..., 4191, 4191, 4191]]), array([[4191, 4191, 4191, ..., 7818, 7818, 7818]]), array([[7818, 7818, 7818, ..., 8117, 8117, 8117]])]


In [9]:
# LBSN2vec
dim_emb = 128
num_epoch = 1
num_threads =  4
K_neg = 10
win_size = 10
learning_rate = 0.001


embs_ini = (np.random.uniform(size=(num_node_total,dim_emb))-0.5)/dim_emb 
temp = np.sum(np.power(embs_ini,2),axis=1)
embs_len = np.power(temp,0.5)
den = embs_len
for i in range(dim_emb-1):
    den=np.vstack((den,embs_len))
den=den.T
#print(den.shape)
#print(embs_ini.shape)
embs_ini = np.divide(embs_ini,den)

mobility_ratio = 0.2

In [10]:
import learn_LBSN2Vec_embedding

embs = learn_LBSN2Vec_embedding.driver_fn(walks.T,user_checkins, user_checkins_counter,embs_ini.T,learning_rate, 
                                K_neg,neg_sam_table_social, win_size, neg_sam_table_mobility_norm,
                                num_epoch, mobility_ratio);

this is num_w 40240
this is num_wl 80
[[   1    2    3 ... 4022 4023 4024]
 [2842   30    3 ... 1990 3323 2823]
 [ 361 1028  215 ... 1533 2430 4024]
 ...
 [3714  268 2428 ...  794 3968  712]
 [1892   84 2620 ...  791 3105 2258]
 [3992 1110 2428 ...  626 3968  712]]
this is num_wl: 0 and num_w 0
[[ 0.07082835  0.1003345  -0.12321929 ... -0.05883234 -0.0417263
  -0.13352464]
 [-0.10609174 -0.13593837  0.11196703 ...  0.08310434  0.10682249
   0.11734305]
 [-0.07149525 -0.07324755 -0.12420157 ...  0.15691299  0.06397323
  -0.01393344]
 ...
 [-0.09685665 -0.11361942  0.14725336 ...  0.10498914 -0.10002669
  -0.14038218]
 [-0.11082957 -0.09355831 -0.08798406 ... -0.09127431  0.06024685
   0.09820616]
 [-0.09633247  0.07420453  0.02995889 ...  0.01916328  0.14586387
  -0.09780114]]
this is num_wl: 1 and num_w 0
[[ 0.0708411   0.1003345  -0.12321929 ... -0.05883234 -0.0417263
  -0.13352464]
 [-0.10610984 -0.13593837  0.11196703 ...  0.08310434  0.10682249
   0.11734305]
 [-0.07140762 -0.07324

this is num_wl: 28 and num_w 0
[[ 0.07082542  0.1003345  -0.12295797 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.1060922  -0.13593837  0.11207624 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07127815 -0.07324755 -0.12426128 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.09715385 -0.11361942  0.14723583 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.11085931 -0.09355831 -0.08798365 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09611496  0.07420453  0.02991589 ...  0.01916328  0.14581575
  -0.09780114]]
this is num_wl: 29 and num_w 0
[[ 0.07082542  0.1003345  -0.12295797 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.1060922  -0.13593837  0.11207624 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07127815 -0.07324755 -0.12426128 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.09715385 -0.11361942  0.14723583 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.11085931 -0.09355831 -0.08798365 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09611496  0.07420453  0.0299158

this is num_wl: 48 and num_w 0
[[ 0.0707886   0.1003345  -0.12260074 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.10600435 -0.13593837  0.11195339 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07128975 -0.07324755 -0.12367564 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.09687946 -0.11361942  0.14741534 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.11091648 -0.09355831 -0.08724734 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09612259  0.07420453  0.02995973 ...  0.01916328  0.14581575
  -0.09780114]]
this is num_wl: 49 and num_w 0
[[ 0.0707886   0.1003345  -0.12260074 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.10600435 -0.13593837  0.11195339 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07128975 -0.07324755 -0.12367564 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.09687946 -0.11361942  0.14741534 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.11091648 -0.09355831 -0.08724734 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09612259  0.07420453  0.0299597

  -0.09780114]]
this is num_wl: 72 and num_w 0
[[ 0.0707886   0.1003345  -0.12260074 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.10600435 -0.13593837  0.11195339 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07128975 -0.07324755 -0.12367564 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.09687946 -0.11361942  0.14741534 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.11091648 -0.09355831 -0.08724734 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09612259  0.07420453  0.02995973 ...  0.01916328  0.14581575
  -0.09780114]]
this is num_wl: 73 and num_w 0
[[ 0.0707886   0.1003345  -0.12260074 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.10600435 -0.13593837  0.11195339 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07128975 -0.07324755 -0.12367564 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.09687946 -0.11361942  0.14741534 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.11091648 -0.09355831 -0.08724734 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09612259  0.074

this is num_wl: 17 and num_w 1
[[ 0.07100902  0.10000132 -0.12260074 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.10609142 -0.13600717  0.11195339 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07129269 -0.07384178 -0.12367564 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.0970244  -0.11418297  0.14741534 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.11110205 -0.09369898 -0.08724734 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09648169  0.07440539  0.02995973 ...  0.01916328  0.14581575
  -0.09780114]]
this is num_wl: 18 and num_w 1
[[ 0.07100902  0.10000132 -0.12260074 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.10609142 -0.13600717  0.11195339 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07129269 -0.07384178 -0.12367564 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.0970244  -0.11418297  0.14741534 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.11110205 -0.09369898 -0.08724734 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09648169  0.07440539  0.0299597

this is num_wl: 33 and num_w 1
[[ 0.07154889  0.10000132 -0.12260074 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.10526599 -0.13600717  0.11195339 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07180842 -0.07384178 -0.12367564 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.09739631 -0.11418297  0.14741534 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.11189297 -0.09369898 -0.08724734 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09635716  0.07440539  0.02995973 ...  0.01916328  0.14581575
  -0.09780114]]
this is num_wl: 34 and num_w 1
[[ 0.07154889  0.10000132 -0.12260074 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.10526599 -0.13600717  0.11195339 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07180842 -0.07384178 -0.12367564 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.09739631 -0.11418297  0.14741534 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.11189297 -0.09369898 -0.08724734 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09635716  0.07440539  0.0299597

[[ 0.07143697  0.10000132 -0.12260074 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.10515479 -0.13600717  0.11195339 ...  0.08310434  0.10690208
   0.11734305]
 [-0.0718235  -0.07384178 -0.12367564 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.09756403 -0.11418297  0.14741534 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.1118808  -0.09369898 -0.08724734 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09629732  0.07440539  0.02995973 ...  0.01916328  0.14581575
  -0.09780114]]
this is num_wl: 58 and num_w 1
[[ 0.07153821  0.10000132 -0.12260074 ... -0.05883234 -0.04158259
  -0.13352464]
 [-0.10508476 -0.13600717  0.11195339 ...  0.08310434  0.10690208
   0.11734305]
 [-0.07177038 -0.07384178 -0.12367564 ...  0.15691299  0.06389621
  -0.01393344]
 ...
 [-0.09755141 -0.11418297  0.14741534 ...  0.10498914 -0.10003634
  -0.14038218]
 [-0.1117623  -0.09369898 -0.08724734 ... -0.09127431  0.06009768
   0.09820616]
 [-0.09618135  0.07440539  0.02995973 ...  0.01916328  0.14581575
 

this is num_wl: 8 and num_w 2
[[ 0.07145929  0.10000132 -0.12345527 ... -0.05883234 -0.04158259
  -0.13347345]
 [-0.10507883 -0.13600717  0.11407598 ...  0.08310434  0.10690208
   0.11735563]
 [-0.07177196 -0.07384178 -0.12280024 ...  0.15691299  0.06389621
  -0.01396229]
 ...
 [-0.09762704 -0.11418297  0.14922435 ...  0.10498914 -0.10003634
  -0.14023277]
 [-0.11176641 -0.09369898 -0.08442803 ... -0.09127431  0.06009768
   0.09814517]
 [-0.0962013   0.07440539  0.03117664 ...  0.01916328  0.14581575
  -0.0977609 ]]
this is num_wl: 9 and num_w 2
[[ 0.07145929  0.10000132 -0.1235075  ... -0.05883234 -0.04158259
  -0.13347345]
 [-0.10507883 -0.13600717  0.11398842 ...  0.08310434  0.10690208
   0.11735563]
 [-0.07177196 -0.07384178 -0.12288397 ...  0.15691299  0.06389621
  -0.01396229]
 ...
 [-0.09762704 -0.11418297  0.14935661 ...  0.10498914 -0.10003634
  -0.14023277]
 [-0.11176641 -0.09369898 -0.08432898 ... -0.09127431  0.06009768
   0.09814517]
 [-0.0962013   0.07440539  0.03120442 

[[ 0.07089777  0.10000132 -0.12361384 ... -0.05883234 -0.04158259
  -0.13347345]
 [-0.10624253 -0.13600717  0.1136545  ...  0.08310434  0.10690208
   0.11735563]
 [-0.07179784 -0.07384178 -0.12306315 ...  0.15691299  0.06389621
  -0.01396229]
 ...
 [-0.09807185 -0.11418297  0.14982505 ...  0.10498914 -0.10003634
  -0.14023277]
 [-0.11128092 -0.09369898 -0.08390748 ... -0.09127431  0.06009768
   0.09814517]
 [-0.09611104  0.07440539  0.03147576 ...  0.01916328  0.14581575
  -0.0977609 ]]
this is num_wl: 30 and num_w 2
[[ 0.07089777  0.10000132 -0.12361384 ... -0.05883234 -0.04158259
  -0.13347345]
 [-0.10624253 -0.13600717  0.1136545  ...  0.08310434  0.10690208
   0.11735563]
 [-0.07179784 -0.07384178 -0.12306315 ...  0.15691299  0.06389621
  -0.01396229]
 ...
 [-0.09807185 -0.11418297  0.14982505 ...  0.10498914 -0.10003634
  -0.14023277]
 [-0.11128092 -0.09369898 -0.08390748 ... -0.09127431  0.06009768
   0.09814517]
 [-0.09611104  0.07440539  0.03147576 ...  0.01916328  0.14581575
 

KeyboardInterrupt: 

In [16]:
neg_sam_table_mobility = neg_sam_table_mobility_norm
neg_sam_table_mobility1 = neg_sam_table_mobility[0][0]
table_size_mobility1 = neg_sam_table_mobility1.shape
table_size_mobility1[0]

100502